### Pre-defined Method

In [1]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import re

def test_compute_metrics(label, pred):
    MAE = []

    for i in range(5):
        if pred[i][-1] == '.':
            pred[i] = pred[i][:-1]
        
        MAE.append(1 - mean_absolute_error([float(label[i])], [float(pred[i])]))
    
    return MAE

### Zero-shot LLM Inference

In [ ]:
# import pandas as pd

# FI_test = pd.read_csv("/home/visi8747/juphome/2025_KETI_CORPUS_TXT/split_output_combined/output_combined_test.csv")  # 데이터 불러오기 (First Impression V2 데이터의 Test set 불러오기)


In [2]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
logging.disable(logging.INFO) 
logging.disable(logging.WARNING)

FI_test = pd.read_csv("/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv")  # 데이터 불러오기 (First Impression V2 데이터의 Test set 불러오기)

MODEL_DIR = "/home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction"
ORIGIN_MODEL_DIR = "LGAI-EXAONE/EXAONE-3.5-32B-Instruct"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.num_parameters())  # 모델 사이즈 출력; KoSOLAR : 10,804,924,416 (10.8B)

tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token
#streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

# genconfig = GenerationConfig(  # Decoding/Sampling Strategy
#     max_new_tokens=50, # 최대 생성 토큰, 1024
#     do_sample=True, 
#     min_new_tokens = 15,
#     temperature=0.9,
#     repetition_penalty=1.3,
#     top_k=50,
#     top_p=0.92,
#     eos_token_id=model.config.eos_token_id
# )

genconfig = GenerationConfig(
    max_new_tokens=150,
    do_sample=False,
    temperature=0.0,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)


result_df = pd.DataFrame(columns=["Text", "Inference"])  # 결과를 저장할 데이터프레임

for content in tqdm(FI_test['script'].to_list()):
    user = content
    inputs = tokenizer.encode(zero_shot2 + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # 입력 데이터 토크나이징
    outputs = model.generate(inputs, 
                             generation_config = genconfig)

    # Post-Processing
    out_text = tokenizer.decode([el.item() for el in outputs[0]])  # output 숫자 값 -> 텍스트로 바꾸기
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")  # output에서 input 빼기

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})  # 만들어진 출력 저장하기
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

/home/visi8747/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/I

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

7818448896


100%|██████████| 8337/8337 [7:40:22<00:00,  3.31s/it]  


In [3]:
result_df['Inference'].to_list()

['0.5520833333333334, C: 0.7916666666666666, E: 0.7604166666666666, A: 0.3958333333333333, N: 0.5625][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.7395833333333334, A: 0.8541666666666666, N: 0.5625][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.71875, A: 0.3541666666666667, N: 0.5625][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.71875, A: 0.4791666666666667, N: 0.59375][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.7395833333333334, A: 0.5833333333333334, N: 0.8333333333333334][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.71875, A: 0.4791666666666667, N: 0.59375][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.7395833333333334, A: 0.5104166666666666, N: 0.8541666666666666][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.7604166666666666, A: 0.3958333333333333, N: 0.5625][|endofturn|]',
 '0.5520833333333334, C: 0.7916666666666666, E: 0.7604166666666666, A: 

In [4]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [5]:
score

[['0.5520833333333334',
  '0.7916666666666666',
  '0.7604166666666666',
  '0.3958333333333333',
  '0.5625'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.7395833333333334',
  '0.8541666666666666',
  '0.5625'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.71875',
  '0.3541666666666667',
  '0.5625'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.71875',
  '0.4791666666666667',
  '0.59375'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.7395833333333334',
  '0.5833333333333334',
  '0.8333333333333334'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.71875',
  '0.4791666666666667',
  '0.59375'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.7395833333333334',
  '0.5104166666666666',
  '0.8541666666666666'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.7604166666666666',
  '0.3958333333333333',
  '0.5625'],
 ['0.5520833333333334',
  '0.7916666666666666',
  '0.7604166666666666',
  '0.84375',
  '0.5625'],
 ['0.5520833333333334',
  '0.79166666

In [6]:
import pandas as pd

FI_test = pd.read_csv(
"/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv",
    index_col=0
)

FI_test['label'] = FI_test.apply(
    lambda r: [
        float(r['openness']),
        float(r['conscientiousness']),
        float(r['extraversion']),
        float(r['agreeableness']),
        float(r['neuroticism'])
    ],
    axis=1
)

In [7]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [8]:
error_idx

[]

In [9]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['script'][error_idx].to_list()):
        user = data
     #   conversation = [{'role': 'system', 'content': zero_shot2}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(zero_shot2 + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)

In [10]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae

(array([0.8735492 , 0.80805948, 0.83001161, 0.8293429 , 0.79575583]),
 np.float64(0.8273438037842491))

--------------------

### One-shot LLM Inference (Kernel Restart Required for GPU Memory)

In [2]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
import pandas as pd
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

FI_test = pd.read_csv("/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv")  # 데이터 불러오기 (First Impression V2 데이터의 Test set 불러오기)

MODEL_DIR = "/home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction"
ORIGIN_MODEL_DIR = "LGAI-EXAONE/EXAONE-3.5-32B-Instruct"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.config.eos_token_id)
print(model.num_parameters())  # KoSOLAR : 10,804,924,416 (10.8B)
tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

one_shot_text = "아메리카노랑 이제 카페니까 네"

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

ko_prompt_one_shot = zero_shot2 + '문장: "' + one_shot_text + '",\n 결과- [O: 0.5, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n'

# genconfig = GenerationConfig(
#     max_new_tokens=50, 
#     do_sample=False, 
#     min_new_tokens = 15,
#     temperature=0.0,
#     repetition_penalty=1.3,
#  #   no_repeat_ngram_size=3,
#     top_k=50,
#     top_p=0.92,
#     eos_token_id=model.config.eos_token_id
# )

genconfig = GenerationConfig(
    max_new_tokens=150,
    do_sample=False,
    temperature=0.0,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)


result_df = pd.DataFrame(columns=["Text", "Inference"])

cnt = 0

for content in tqdm(FI_test['script'].to_list()):
    user = content
    inputs = tokenizer.encode(ko_prompt_one_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  


    outputs = model.generate(inputs, 
                             generation_config = genconfig)
    out_text = tokenizer.decode([el.item() for el in outputs[0]])
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

/home/visi8747/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  Y
The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/I

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

361
7818448896


100%|██████████| 8337/8337 [11:52:04<00:00,  5.12s/it]  


In [3]:
result_df['Inference'].to_list()

['0.5, C: 0.7395833333333334, E: 0.5416666666666666, A: 0.4791666666666667, N: 0.5833333333333334] \n문장: "그러면 1위부터 일단 얘기를 해볼까요? 그래서 1명씩 1위를 얘기를 한 다음에 거기서 얘기를 해서 1 2 3 4위를 정하고 그렇게 하는 게 조금 편할 것 같아요',
 '0.5, C: 0.7395833333333334, E: 0.5520833333333334, A: 0.6770833333333334, N: 0.3541666666666667] \n문장: "저는 아이유를 1위로 했고 그다음에 2위를 이제 노라조로 했습니다.",\n 결과- [O: 0.5, C: 0.8020833333333',
 '0.5, C: 0.7395833333333334, E: 0.5520833333333334, A: 0.40625, N: 0.7708333333333334][|endofturn|]',
 '0.5, C: 0.5520833333333334, E: 0.5416666666666666, A: 0.6041666666666666, N: 0.6041666666666666] \n문장: "저는 11위요",\n 결과- [O: 0.5, C: 0.65625, E: 0.5520833333333334, A:',
 '0.5, C: 0.8333333333333334, E: 0.5416666666666666, A: 0.3541666666666667, N: 0.5729166666666666] \n문장: "박나래가 2위를 했고 그다음에 3위를 이제 누구로 할까요?",\n 결과- [O: 0.5, C: 0.833333333333333',
 '0.5, C: 0.65625, E: 0.5520833333333334, A: 0.6041666666666666, N: 0.3020833333333333] \n문장: "나영석 박나래 노라조 3위 4위 5위"\n 결과- [O: 0.5, C: 0.7083333333333334, E: 0.6041666

In [4]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [5]:
score

[['0.5',
  '0.7395833333333334',
  '0.5416666666666666',
  '0.4791666666666667',
  '0.5833333333333334'],
 ['0.5',
  '0.7395833333333334',
  '0.5520833333333334',
  '0.6770833333333334',
  '0.3541666666666667',
  '0.5',
  '0.8020833333333'],
 ['0.5',
  '0.7395833333333334',
  '0.5520833333333334',
  '0.40625',
  '0.7708333333333334'],
 ['0.5',
  '0.5520833333333334',
  '0.5416666666666666',
  '0.6041666666666666',
  '0.6041666666666666',
  '0.5',
  '0.65625',
  '0.5520833333333334'],
 ['0.5',
  '0.8333333333333334',
  '0.5416666666666666',
  '0.3541666666666667',
  '0.5729166666666666',
  '0.5',
  '0.833333333333333'],
 ['0.5',
  '0.65625',
  '0.5520833333333334',
  '0.6041666666666666',
  '0.3020833333333333',
  '0.5',
  '0.7083333333333334',
  '0.6041666666'],
 ['0.5',
  '0.6042',
  '0.5208',
  '0.6875',
  '0.3437',
  '0.5',
  '0.65625',
  '0.5417',
  '0.67791667',
  '0.32291667'],
 ['0.5',
  '0.5520833333333334',
  '0.5416666666666666',
  '0.6041666666666666',
  '0.40625',
  '0.5',


In [6]:
import pandas as pd

FI_test = pd.read_csv(
"/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv",
    index_col=0
)

FI_test['label'] = FI_test.apply(
    lambda r: [
        float(r['openness']),
        float(r['conscientiousness']),
        float(r['extraversion']),
        float(r['agreeableness']),
        float(r['neuroticism'])
    ],
    axis=1
)

In [7]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [8]:
error_idx

[]

In [9]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['script'][error_idx].to_list()):
        user = data
        inputs = tokenizer.encode(ko_prompt_one_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda") 
        outputs = model.generate(inputs, 
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)

In [10]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae

(array([0.84670496, 0.86370714, 0.83873881, 0.89446589, 0.83452877]),
 np.float64(0.8556291129242568))

------------

### five-shot LLM Inference (Kernel Restart Required for GPU Memory)

In [2]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
import pandas as pd
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

FI_test = pd.read_csv("/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv")  # 데이터 불러오기 (First Impression V2 데이터의 Test set 불러오기)

MODEL_DIR = "/home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction"
ORIGIN_MODEL_DIR = "LGAI-EXAONE/EXAONE-3.5-32B-Instruct"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.config.eos_token_id)
print(model.num_parameters())  # KoSOLAR : 10,804,924,416 (10.8B)

tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

few_shot_text0 = "저 펭수 2순위로 했어요. 저희"
few_shot_text1 = "대구에는 뭐 많지 대구는 뭐 알지"
few_shot_text2 = "일단 아침 8시에 인천공항에 도착하는 걸로 내일 아랑 한복"
few_shot_text3 = "9위가 이수근이었어요. 남은 이제 5등은 누구로 하셨나요?"
few_shot_text4 = "야간으로 놀러 다녀야 될 거 아니에요 그러면 거기 말고"

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'

ko_prompt_few_shot = zero_shot2 + '문장: "' + few_shot_text0 + '",\n 결과- [O: 0.5000, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n' + \
                    '문장: "' + few_shot_text1 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n' + \
                    '문장: "' + few_shot_text2 + '",\n 결과- [O: 0.5, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n' + \
                    '문장: "' + few_shot_text3 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n' + \
                    '문장: "' + few_shot_text4 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n'


# genconfig = GenerationConfig(
#     max_new_tokens=50, 
#     do_sample=True, 
#     min_new_tokens = 15,
#     temperature=0.9,
#     repetition_penalty=1.3,
#     top_k=50,
#     top_p=0.92,
#     eos_token_id=model.config.eos_token_id
# )


genconfig = GenerationConfig(
    max_new_tokens=150,
    do_sample=False,
    temperature=0.0,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)

result_df = pd.DataFrame(columns=["Text", "Inference"])

cnt = 0

for content in tqdm(FI_test['script'].to_list()):
    user = content
    inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda") 

    outputs = model.generate(inputs, 
                             generation_config = genconfig)
    
    out_text = tokenizer.decode([el.item() for el in outputs[0]])
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

/home/visi8747/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  ㅛ
The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/I

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

361
7818448896


100%|██████████| 8337/8337 [13:14:07<00:00,  5.72s/it]  


In [3]:
result_df['Inference'].to_list()

['0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n문장: "그러면 1위부터 얘기를 해볼까요? 그래서 1명씩 1위를 얘기를 한 다음에 거기서 얘기를 해서 1 2 3 4위를 정하고 그렇게 하는 게 조금 편할 것 같아요. 저도 이제 아이유를 첫 번째로 했는데 아까 1 2번 분이 말씀해 주신 것처럼 다양한 연령층을 즐길 수 있고 팬층이 굉장히 넓기 때문에",\n 결과- [',
 '0.5, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n문장: "저는 1위로 아이유를 선택을 했습니다. 그럼 팀 순위 1위를 아이유로 하시고 그다음에 2위를",\n 결과- [O: 0.5, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n문장: "저는 1위로 아이유를 선택을 했습니다. 그럼 팀',
 '0.5520, C: 0.7188, E: 0.6354, A: 0.5520, N: 0.4583] \n문장: "저는 10위로 펭수를 했습니다.",\n 결과- [O: 0.5520, C: 0.7188, E: 0.6354, A: 0.5520, N: 0.4583] \n문장: "저는 11위로 보아를 했습니다.",\n 결과- [O: 0.5520,',
 '0.5, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n문장: "저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위요. 저는 10위',
 '0.5, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n문장: "3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위가 박나래였어요. 3위

In [4]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [5]:
score

[['0.5729', '0.7188', '0.6771', '0.4583', '0.5208'],
 ['0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292',
  '0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292'],
 ['0.5520',
  '0.7188',
  '0.6354',
  '0.5520',
  '0.4583',
  '0.5520',
  '0.7188',
  '0.6354',
  '0.5520',
  '0.4583',
  '0.5520'],
 ['0.5', '0.6354', '0.4583', '0.5417', '0.2292'],
 ['0.5', '0.6354', '0.4583', '0.5417', '0.2292'],
 ['0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292',
  '0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292'],
 ['0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292',
  '0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292'],
 ['0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292',
  '0.5',
  '0.6354',
  '0.4583',
  '0.5417',
  '0.2292'],
 ['0.5520',
  '0.7188',
  '0.5520',
  '0.5625',
  '0.4583',
  '0.5520',
  '0.7188',
  '0.5520',
  '0.5625',
  '0.4583'],
 ['0.5729',
  '0.7188',
  '0.6771',
  '0.4583',
  '0.5208',
  '0.5',
  '0.6354',
  '0.4583',
  '0.'],
 ['0.5',
  '0.5520',
  '0.4583'

In [6]:
import pandas as pd

FI_test = pd.read_csv(
    "/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv",
    index_col=0
)

FI_test['label'] = FI_test.apply(
    lambda r: [
        float(r['openness']),
        float(r['conscientiousness']),
        float(r['extraversion']),
        float(r['agreeableness']),
        float(r['neuroticism'])
    ],
    axis=1
)

In [7]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [8]:
error_idx

[]

In [9]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['script'][error_idx].to_list()):
        user = data
     #   conversation = [{'role': 'system', 'content': en_prompt}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)

In [10]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)
 
factor_mae, avg_mae

(array([0.85414427, 0.87255   , 0.82769786, 0.91408082, 0.81543308]),
 np.float64(0.8567812055979156))

### ETC

In [ ]:
p1 = "너는 지금부터 문장에 대해 BIG 5 성격 검사의 결과를 보여주는 전문 계산기야. 너는 전문가로써 BIG 5에 대한 정보를 충분히 알고있고 \
주어진 문장에 대해 5가지 요소를 가지는 BIG 5 성격 검사로 개성을 평가해야만 해. \
이 때, 문장이 주어지면 각각의 요소(개방성, 성실성, 우호성, 외향성, 신경증)에 대해 0부터 1사이의 값을 가지도록 숫자로 평가해야하고, \
모든 요소에 대해 최대한 세밀하고 정확하게 값을 출력해야 해. \
결과를 출력할 때, BIG 5에 대한 설명이나 이유에 대해서는 필요 없이 숫자 값만 보여줘. <요소 - 숫자> 형식으로 출력해.\
문장 : 오늘 워크샵은 정말 기대돼!>"

p2 = "당신은 문장에 대해 BIG 5 성격 검사의 결과를 보여주는 전문가 입니다. 주어진 문장을 읽고, 다음과 같은 OCEAN 요소들을 각각 0에서 1 사이의 값으로 나타내세요. O(Openness): 개방성, C(Conscientiousness): 성실성, \
E(Extraversion): 외향성, A(Agreeableness): 친화성, N(Neuroticism): 신경성, 문장을 이용하여 각 요소의 점수를 계산할 때, 다음과 같은 접근을 사용할 수 있습니다: \
Openness (O): 문장에 포함된 창의성, 호기심, 복잡성에 대한 언급 등을 고려합니다. Conscientiousness (C): 문장에서 나타나는 계획성, 철저함, 조직성을 고려합니다. \
Extraversion (E): 문장에서 외향적인 특성, 사회적 활동에 대한 언급을 고려합니다. Agreeableness (A): 문장에서 타인에 대한 관심, 협동성, 친화력을 고려합니다. \
Neuroticism (N): 문장에 포함된 감정적 불안, 긴장, 우울 등을 고려합니다. \
이러한 기준을 이용하여 주어진 문장을 분석하여 각 요소에 대한 0에서 1 사이의 값을 계산해보세요. \
계산된 값은 다음의 예시처럼 다섯 개의 요소에 대한 배열로 표현되어야 합니다. \
예시) 다른 방에서 미친 듯이 달려가서 없는지 확인했는데 없는 것 같았어요 이제 잠을 충분히 못 자서 가위를 어딘가로 옮겼을 수도 있지만 맹세컨대 - [0.8222, 0.6699, 0.4860, 0.6458, 0.6813]"

In [ ]:
base_prompt = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'
one_shot_text = "현재 가장 좋아하는 책 사실 몇 달 전에 서점에서 이 책을 샀는데 읽을 기회가 없었는데 읽기 시작했어요 한번 시작하면 내려놓고 싶지 않은 책 조이스 마이어의 <용감하게 살기>라는 책이에요."

zero = base_prompt + '문장: "' + "<EXAMPLE SENTENCE OF FI_TEST DATASET>" + '",\n 결과- [O: '

one = base_prompt + '문장: "' + one_shot_text + '",\n 결과- [O: 0.6333, C: 0.7184, E: 0.4019, A: 0.4176, N: 0.5313] \n' + '문장: "' + "<EXAMPLE SENTENCE OF FI_TEST DATASET>" + '",\n 결과- [O: '

print("Zero shot Input: ", "\n", zero )
print("\n\n" ,"========================="*15, "\n\n")
print("One shot Input: ", "\n", one)

### prompt example mining

In [ ]:
import pandas as pd

val = pd.read_csv("/data/visi2/FI/KETI_test_from_ver0.4.csv",
                 index_col=0)

In [ ]:
val.describe().T

In [ ]:
open_high = val[val['openness'] == val['openness'].max()]
open_low = val.sort_values(by='openness')
cons_high = val[val['conscientiousness'] == val['conscientiousness'].max()]
cons_low = val[val['conscientiousness'] == val['conscientiousness'].min()]
extra_high = val[val['extraversion'] == val['extraversion'].max()]
extra_low = val.sort_values(by='extraversion')
agree_high = val.sort_values(by='agreeableness',ascending=False)
agree_low = val.sort_values(by='agreeableness')
neuro_high = val[val['neuroticism'] == val['neuroticism'].max()]
neuro_low = val.sort_values(by='neuroticism')

In [ ]:
neuro_low

In [ ]:
res = {
    open_high['transcription'].to_list()[0] : open_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    open_low['transcription'].to_list()[1] : open_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    cons_high['transcription'].to_list()[0] : cons_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    cons_low['transcription'].to_list()[0] : cons_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    extra_high['transcription'].to_list()[0] : extra_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    extra_low['transcription'].to_list()[1] : extra_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    agree_high['transcription'].to_list()[1] : agree_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    agree_low['transcription'].to_list()[1] : agree_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    neuro_high['transcription'].to_list()[0] : neuro_high[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
    neuro_low['transcription'].to_list()[1] : neuro_low[["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]].iloc[0],
}

len(res), res

### Ten-Shot Inference

In [2]:
import torch
import pandas as pd
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import logging
import pandas as pd
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

FI_test = pd.read_csv("/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv")  # 데이터 불러오기 (First Impression V2 데이터의 Test set 불러오기)

MODEL_DIR = "/home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction"
ORIGIN_MODEL_DIR = "LGAI-EXAONE/EXAONE-3.5-32B-Instruct"  # 모델 이름
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")

print(model.config.eos_token_id)
print(model.num_parameters())  

# KoSOLAR : 10,804,924,416 (10.8B)

tokenizer = AutoTokenizer.from_pretrained(ORIGIN_MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token

few_shot_text0 = "그래서 그래서 바짝 놀자는 거야. 근데 한국 사람이잖아"
few_shot_text1 = "극 J 엑셀 사용하죠"
few_shot_text2 = "끝 아 로봇"
few_shot_text3 = "좋은 호텔이라든가 물배요? 물배?"
few_shot_text4 = "네. 그리고 밥을 먹이고요. 이제 명동 광화문 닭갈비를 먹이고"
few_shot_text5 = "맞습니다. 어"
few_shot_text6 = "선재 응"
few_shot_text7 = "한국 어디 사셨는데요? 용인?"
few_shot_text8 = "푸바오 보고 왔었는데 나무 있잖아 나무 위에 올라가서 안 일어나요."
few_shot_text9 = "음료수 등등등 음료수 옆에 있으니까 딱 맞을 것 같아. 그것도 괜찮을 것 같고 붕어빵"

zero_shot2 = '당신은 문장에 대해 OCEAN 성격 요소 각각에 대한 점수를 알려주는 전문가 입니다. \n\
OCEAN 성격 요소는 다음에 대한 사항을 확인합니다. O(Openness), C(Conscientiousness), E(Extraversion), A(Agreeableness), N(Neuroticism) \n\
주어진 문장을 읽고, 문장에서 나타나는 OCEAN 성격 요소들을 분석하여 각각의 요소에 0에서 1 사이의 점수를 매기세요. \n\
점수는 문장에서 해당 요소가 얼마나 잘 드러나는지를 기반으로 하며, 0에 가까울수록 전혀 드러나지 않음을, 1에 가까울수록 매우 강하게 드러남을 의미합니다. \n\
다음의 예시와 같이 출력 값으로 주어진 문장에 대한 OCEAN 성격 요소 5가지를 각각의 점수로 나타내어야합니다. [O: 0.6722, C: 0.8322, E: 0.4860, A: 0.5298, N: 0.9323] \n\
'
 
ko_prompt_few_shot = zero_shot2 + '문장: "' + few_shot_text0 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n' + \
                    '문장: "' + few_shot_text1 + '",\n 결과- [O: 0.5000, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n' + \
                    '문장: "' + few_shot_text2 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n' + \
                    '문장: "' + few_shot_text3 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208]\n' + \
                    '문장: "' + few_shot_text4 + '",\n 결과- [O: 0.7292, C: 0.5313, E: 0.4896, A: 0.6875, N: 0.4792] \n' + \
                    '문장: "' + few_shot_text5 + '",\n 결과- [O: 0.5000, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n' + \
                    '문장: "' + few_shot_text6 + '",\n 결과- [O: 0.7292, C: 0.5313, E: 0.4896, A: 0.6875, N: 0.4792] \n' + \
                    '문장: "' + few_shot_text7 + '",\n 결과- [O: 0.7292, C: 0.5313, E: 0.4896, A: 0.6875, N: 0.4792] \n' + \
                    '문장: "' + few_shot_text8 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n' + \
                    '문장: "' + few_shot_text9 + '",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n'

# genconfig = GenerationConfig(
#     max_new_tokens=50, 
#     do_sample=True, 
#     min_new_tokens = 15,
#     temperature=0.9, 
#     repetition_penalty=1.3,
#     top_k=50,
#     top_p=0.92,
#     eos_token_id=model.config.eos_token_id
# )

genconfig = GenerationConfig(
    max_new_tokens=150,
    do_sample=False,
    temperature=0.0,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)



result_df = pd.DataFrame(columns=["Text", "Inference"])

cnt = 0

for content in tqdm(FI_test['script'].to_list()):
    user = content
    inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda") 

    outputs = model.generate(inputs, 
                             generation_config = genconfig)
    
    out_text = tokenizer.decode([el.item() for el in outputs[0]])
    out_text = out_text.replace(tokenizer.decode(inputs[0]), "")

    tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
    result_df = pd.concat([result_df, tmp_df])

# KULLM, KoSOLAR (SOLAR-10.7B) : 약 21GB GPU 메모리 소모

/home/visi8747/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  Y
The repository /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction contains custom code which must be executed to correctly load the model. You can inspect the repository content at /home/visi8747/juphome/model_PersonalityAI/Instruction_tuning/Bllossom-8B-ft-instruction .
 You can inspect the repository content at https://hf.co//home/visi8747/juphome/model_PersonalityAI/I

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

361
7818448896


100%|██████████| 8337/8337 [12:51:09<00:00,  5.55s/it]  


In [3]:
result_df['Inference'].to_list()

['0.6771, C: 0.7188, E: 0.5729, A: 0.5208, N: 0.4583] \n문장: "근데 이제 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를 해볼까요? 1위부터 얘기를',
 '0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n문장: "근데 저는 아이유를 1위로 했습니다.",\n 결과- [O: 0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n문장: "근데 저는 아이유를 1위로 했습니다.",\n 결과- [O: 0.5729',
 '0.6771, C: 0.7188, E: 0.5729, A: 0.5208, N: 0.4583] \n문장: "근데 이제 1위를 아이유로 했는데 아이유를 1위로 한 이유는 솔직히 아이유 노래 중에 모르는 노래가 거의 없잖아요. 그리고 이제 아이유가 이제 콘서트를 하면은 거의 1만 명 이상의 관객이 오잖아요. 그래서 이제 아이유를 1위로 했고",\n 결과- [O: 0.6771, C: 0.7188, E: 0.5729',
 '0.5000, C: 0.6354, E: 0.4583, A: 0.5417, N: 0.2292] \n문장: "근데 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저는 11위요. 저',
 '0.5729, C: 0.7188, E: 0.6771, A: 0.4583, N: 0.5208] \n문장: "아"\n 결과- [O: 0.5520, C: 0.6771, E: 0.4583, A: 0.40625, N: 0.5625][|endofturn|]',
 '0.6771, C: 0.7188, E: 0.5729, A: 0.520

In [4]:
score = []
p = re.compile('0[.]\d*')

for content in result_df['Inference'].to_list():
    tmp = p.findall(content)
    score.append(tmp)

In [5]:
score

[['0.6771', '0.7188', '0.5729', '0.5208', '0.4583'],
 ['0.5729',
  '0.7188',
  '0.6771',
  '0.4583',
  '0.5208',
  '0.5729',
  '0.7188',
  '0.6771',
  '0.4583',
  '0.5208',
  '0.5729'],
 ['0.6771',
  '0.7188',
  '0.5729',
  '0.5208',
  '0.4583',
  '0.6771',
  '0.7188',
  '0.5729'],
 ['0.5000', '0.6354', '0.4583', '0.5417', '0.2292'],
 ['0.5729',
  '0.7188',
  '0.6771',
  '0.4583',
  '0.5208',
  '0.5520',
  '0.6771',
  '0.4583',
  '0.40625',
  '0.5625'],
 ['0.6771',
  '0.7188',
  '0.5729',
  '0.5208',
  '0.4583',
  '0.6771',
  '0.7188',
  '0.5729',
  '0.5208'],
 ['0.5729',
  '0.7188',
  '0.6771',
  '0.4583',
  '0.5208',
  '0.5520',
  '0.6771',
  '0.4896',
  '0.5417',
  '0.3646'],
 ['0.5729', '0.7188', '0.6771', '0.4583', '0.5208'],
 ['0.6771',
  '0.7188',
  '0.5729',
  '0.5520',
  '0.5520',
  '0.6771',
  '0.7188',
  '0.5729',
  '0.5520'],
 ['0.6771',
  '0.7188',
  '0.5729',
  '0.5208',
  '0.4583',
  '0.6771',
  '0.7188',
  '0.5729',
  '0.5208',
  '0.4583'],
 ['0.5000',
  '0.6354',
  '0.

In [6]:
import pandas as pd

FI_test = pd.read_csv(
    "/home/visi8747/juphome/2025_KETI_CORPUS_TXT_data/split_output_combined/output_combined_test.csv",
    index_col=0
)

FI_test['label'] = FI_test.apply(
    lambda r: [
        float(r['openness']),
        float(r['conscientiousness']),
        float(r['extraversion']),
        float(r['agreeableness']),
        float(r['neuroticism'])
    ],
    axis=1
)

In [7]:
MAE_arr = []
error_idx = []

for i in range(len(FI_test)):
    try:
        MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
    except Exception as e:
        print(i, e)
        error_idx.append(i)

In [8]:
error_idx

[]

In [9]:
while len(error_idx) !=0:
    re_df = pd.DataFrame(columns=["Text", "Inference"])
    
    for data in tqdm(FI_test['script'][error_idx].to_list()):
        user = data
     #   conversation = [{'role': 'system', 'content': en_prompt}, {'role' : 'user', 'content' : 'Given Sentence: "' + user + '",\n Results- [O: '}]
        inputs = tokenizer.encode(ko_prompt_few_shot + '문장: "' + user + '",\n 결과- [O: ', return_tensors='pt').to("cuda")  # No template
        outputs = model.generate(inputs, 
                    #      streamer=streamer,
                                 generation_config = genconfig)
        
        out_text = tokenizer.decode([el.item() for el in outputs[0]])
        out_text = out_text.replace(tokenizer.decode(inputs[0]), "")
    
        tmp_df = pd.DataFrame({'Text': [user], 'Inference': [out_text]})
        re_df = pd.concat([re_df, tmp_df])

    re_score = []
    p = re.compile('0[.]\d*')
    
    for content in re_df['Inference'].to_list():
        tmp = p.findall(content)
        re_score.append(tmp)
        
    for i, idx in enumerate(error_idx):
        score[idx] = re_score[i]

    MAE_arr = []
    error_idx = []
    
    for i in range(len(FI_test)):
        try:
            MAE_arr.append(test_compute_metrics(FI_test['label'].iloc[i][:5], score[i][:5]))
        except Exception as e:
            print(i, e)
            error_idx.append(i)


In [10]:
factor_mae = np.array(MAE_arr).mean(axis=0)
avg_mae = np.mean(factor_mae)

factor_mae, avg_mae

(array([0.88332267, 0.8492719 , 0.84774339, 0.89279903, 0.84967413]),
 np.float64(0.8645622242757938))